In [1]:
from __future__ import print_function
import torch.nn as nn
import torch
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
from torch.autograd import Variable

In [2]:
batchSize = 64
imageSize = 64

In [4]:
transform = transforms.Compose([transforms.Scale(imageSize), transforms.ToTensor(),transforms.Normalize(
    (0.5,0.5,0.5),(0.5,0.5,0.5))])

C:\Users\Windows10x64\Anaconda3\lib\site-packages\torchvision\transforms\transforms.py:219: UserWarning: The use of the transforms.Scale transform is deprecated, please use transforms.Resize instead.
  warnings.warn("The use of the transforms.Scale transform is deprecated, " +


In [5]:
dataset = dset.CIFAR10(root='./data', download=True, transform=transform)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=batchSize,shuffle=True,num_workers=2)

Files already downloaded and verified


In [6]:
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        m.weight.data.normal_(0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        m.weight.data.normal_(1.0, 0.02)
        m.bias.data.fill_(0)

In [11]:
# Defining the generator
class G(nn.Module):
    
    def __init__(self):
        super(G, self).__init__()
        self.main = nn.Sequential(
            nn.ConvTranspose2d(100, 512, 4, 1, 0, bias=False),
            nn.BatchNorm2d(512),
            nn.ReLU(True),
            nn.ConvTranspose2d(512, 256, 4, 2, 1, bias=False),
            nn.BatchNorm2d(256),
            nn.ReLU(True),
            nn.ConvTranspose2d(256, 128, 4, 2, 1, bias=False),
            nn.BatchNorm2d(128),
            nn.ReLU(True),
            nn.ConvTranspose2d(128, 64, 4, 2, 1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(True),
            nn.ConvTranspose2d(64, 3, 4, 2, 1, bias=False),
            nn.Tanh()
        )
    
    def forward(self, input):
        output = self.main(input)
        return output

In [13]:
#Creating the generator
netG = G()
netG.apply(weights_init)

G(
  (main): Sequential(
    (0): ConvTranspose2d(100, 512, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): ConvTranspose2d(512, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (4): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): ConvTranspose2d(256, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (7): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU(inplace=True)
    (9): ConvTranspose2d(128, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (10): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): ReLU(inplace=True)
    (12): ConvTranspose2d(64, 3, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (13): Tanh()
  )
)

In [14]:
#Defining the descriminator
class D(nn.Module):
    def __init__(self):
        super(D, self).__init__()
        self.main = nn.Sequential(
            nn.Conv2d(3, 64, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, True),
            nn.Conv2d(64, 128, 4, 2, 1, bias=False),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2, True),
            nn.Conv2d(128, 256, 4, 2, 1, bias=False),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2, True),
            nn.Conv2d(256, 512, 4, 2, 1, bias=False),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.2, True),
            nn.Conv2d(512, 1, 4, 1, 0, bias=False),
            nn.Sigmoid()
        )
        
    def forward(self, input):
        output = self.main(input)
        return output.view(-1) #flatten the result of convolution 

In [15]:
#Creating the discriminator
netD = D()
netD.apply(weights_init)

D(
  (main): Sequential(
    (0): Conv2d(3, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (1): LeakyReLU(negative_slope=0.2, inplace=True)
    (2): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (4): LeakyReLU(negative_slope=0.2, inplace=True)
    (5): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (6): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): LeakyReLU(negative_slope=0.2, inplace=True)
    (8): Conv2d(256, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (9): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): LeakyReLU(negative_slope=0.2, inplace=True)
    (11): Conv2d(512, 1, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (12): Sigmoid()
  )
)

In [19]:
#Training GANs
criterion = nn.BCELoss() #binary crossentropy
optimizerD = optim.Adam(netD.parameters(),lr=0.0002,betas=(0.5,0.999))
optimizerG = optim.Adam(netG.parameters(),lr=0.0002,betas=(0.5,0.999))

for epoch in range(25):
    for i, data in enumerate(dataloader,0):
        netD.zero_grad()
        #Training the discriminator with real images
        real, _ = data
        input = Variable(real)
        target = Variable(torch.ones(input.size()[0]))
        output = netD(input)
        errD_real = criterion(output, target)
        
        #Training the discriminator with fake images
        noise = Variable(torch.randn(input.size()[0],100,1,1))
        fake = netG(noise)
        target = Variable(torch.zeros(input.size()[0]))
        output = netD(fake.detach()) #detach because we don't need grad
        errD_fake = criterion(output, target)
        
        #Backpropagation the total error
        errD = errD_real + errD_fake
        errD.backward() 
        optimizerD.step()
        
        #Updating the weights of the generator
        netG.zero_grad()
        target = Variable(torch.ones(input.size()[0]))
        output = netD(fake)
        errG = criterion(output, target)
        errG.backward()
        optimizerG.step()
        
        print('[%d/%d][%d/%d] Loss_D: %.4f Loss_G: %.4f' % (epoch, 25, i, len(dataloader),
                                                            errD.data, errG.data))
        if i%100 == 0:
            vutils.save_image(real,'%s/real_samples.png' % "./results", normalize = True)
            fake = netG(noise)
            vutils.save_image(fake.data, '%s/fake_samples_epoch_%03d.png' % ("./results", epoch), normalize = True)

[0/25][0/782] Loss_D: 1.2772 Loss_G: 7.8039
[0/25][1/782] Loss_D: 0.8596 Loss_G: 5.1092
[0/25][2/782] Loss_D: 1.1560 Loss_G: 5.8480
[0/25][3/782] Loss_D: 0.9882 Loss_G: 6.5699
[0/25][4/782] Loss_D: 1.1813 Loss_G: 6.1885
[0/25][5/782] Loss_D: 0.8064 Loss_G: 7.5477
[0/25][6/782] Loss_D: 0.7593 Loss_G: 6.8635
[0/25][7/782] Loss_D: 0.7462 Loss_G: 8.5677
[0/25][8/782] Loss_D: 0.9836 Loss_G: 7.1548
[0/25][9/782] Loss_D: 0.8836 Loss_G: 10.0326
[0/25][10/782] Loss_D: 0.3098 Loss_G: 7.9895
[0/25][11/782] Loss_D: 0.7526 Loss_G: 10.0808
[0/25][12/782] Loss_D: 0.3749 Loss_G: 8.3990
[0/25][13/782] Loss_D: 0.5983 Loss_G: 10.3001
[0/25][14/782] Loss_D: 0.2801 Loss_G: 8.2950
[0/25][15/782] Loss_D: 0.8862 Loss_G: 13.5053
[0/25][16/782] Loss_D: 0.3613 Loss_G: 10.9123
[0/25][17/782] Loss_D: 0.2371 Loss_G: 7.5487
[0/25][18/782] Loss_D: 1.9867 Loss_G: 17.1233
[0/25][19/782] Loss_D: 0.5822 Loss_G: 16.8607
[0/25][20/782] Loss_D: 0.1535 Loss_G: 11.0658
[0/25][21/782] Loss_D: 0.3149 Loss_G: 8.0898
[0/25][22/78

[0/25][352/782] Loss_D: 0.7924 Loss_G: 5.5433
[0/25][353/782] Loss_D: 0.4962 Loss_G: 3.2728
[0/25][354/782] Loss_D: 0.7662 Loss_G: 5.7790
[0/25][355/782] Loss_D: 0.6943 Loss_G: 2.3509
[0/25][356/782] Loss_D: 1.7584 Loss_G: 7.0059
[0/25][357/782] Loss_D: 1.3770 Loss_G: 4.0292
[0/25][358/782] Loss_D: 0.5177 Loss_G: 4.0557
[0/25][359/782] Loss_D: 0.6121 Loss_G: 4.0061
[0/25][360/782] Loss_D: 0.4381 Loss_G: 4.3747
[0/25][361/782] Loss_D: 0.4954 Loss_G: 4.6765
[0/25][362/782] Loss_D: 0.4310 Loss_G: 5.2762
[0/25][363/782] Loss_D: 0.4178 Loss_G: 3.7040
[0/25][364/782] Loss_D: 0.7265 Loss_G: 6.6201
[0/25][365/782] Loss_D: 0.8932 Loss_G: 3.5476
[0/25][366/782] Loss_D: 0.6691 Loss_G: 6.9100
[0/25][367/782] Loss_D: 0.3824 Loss_G: 5.4711
[0/25][368/782] Loss_D: 0.3768 Loss_G: 4.6608
[0/25][369/782] Loss_D: 0.6532 Loss_G: 6.9795
[0/25][370/782] Loss_D: 0.4079 Loss_G: 4.6449
[0/25][371/782] Loss_D: 0.3294 Loss_G: 5.5021
[0/25][372/782] Loss_D: 0.2673 Loss_G: 5.8141
[0/25][373/782] Loss_D: 0.2524 Los

[0/25][707/782] Loss_D: 0.7978 Loss_G: 4.9676
[0/25][708/782] Loss_D: 0.3983 Loss_G: 4.9544
[0/25][709/782] Loss_D: 0.5072 Loss_G: 3.1612
[0/25][710/782] Loss_D: 0.6225 Loss_G: 4.9905
[0/25][711/782] Loss_D: 0.5475 Loss_G: 3.6569
[0/25][712/782] Loss_D: 0.4639 Loss_G: 3.5028
[0/25][713/782] Loss_D: 0.3369 Loss_G: 4.5349
[0/25][714/782] Loss_D: 0.6077 Loss_G: 3.4761
[0/25][715/782] Loss_D: 0.2524 Loss_G: 4.8446
[0/25][716/782] Loss_D: 0.2462 Loss_G: 4.9279
[0/25][717/782] Loss_D: 0.4025 Loss_G: 4.4348
[0/25][718/782] Loss_D: 0.4615 Loss_G: 4.8326
[0/25][719/782] Loss_D: 0.2388 Loss_G: 4.4783
[0/25][720/782] Loss_D: 0.5113 Loss_G: 3.1060
[0/25][721/782] Loss_D: 0.9395 Loss_G: 7.9945
[0/25][722/782] Loss_D: 0.6654 Loss_G: 6.5505
[0/25][723/782] Loss_D: 0.2166 Loss_G: 4.3535
[0/25][724/782] Loss_D: 0.2725 Loss_G: 4.2227
[0/25][725/782] Loss_D: 0.2582 Loss_G: 5.2176
[0/25][726/782] Loss_D: 0.3162 Loss_G: 5.1219
[0/25][727/782] Loss_D: 0.3583 Loss_G: 4.0720
[0/25][728/782] Loss_D: 0.3674 Los

[1/25][283/782] Loss_D: 0.3612 Loss_G: 2.7501
[1/25][284/782] Loss_D: 0.4322 Loss_G: 2.8809
[1/25][285/782] Loss_D: 0.2967 Loss_G: 3.4371
[1/25][286/782] Loss_D: 0.8397 Loss_G: 4.6556
[1/25][287/782] Loss_D: 0.7599 Loss_G: 2.5154
[1/25][288/782] Loss_D: 0.4271 Loss_G: 3.6903
[1/25][289/782] Loss_D: 0.3142 Loss_G: 4.2501
[1/25][290/782] Loss_D: 0.4842 Loss_G: 2.4706
[1/25][291/782] Loss_D: 0.4150 Loss_G: 3.4894
[1/25][292/782] Loss_D: 0.3550 Loss_G: 4.9360
[1/25][293/782] Loss_D: 0.2290 Loss_G: 4.4603
[1/25][294/782] Loss_D: 0.1841 Loss_G: 3.4098
[1/25][295/782] Loss_D: 0.2557 Loss_G: 4.0514
[1/25][296/782] Loss_D: 0.4670 Loss_G: 3.1204
[1/25][297/782] Loss_D: 0.4510 Loss_G: 3.8209
[1/25][298/782] Loss_D: 0.4122 Loss_G: 3.6673
[1/25][299/782] Loss_D: 0.3743 Loss_G: 2.3226
[1/25][300/782] Loss_D: 0.8453 Loss_G: 6.6155
[1/25][301/782] Loss_D: 1.0426 Loss_G: 3.2667
[1/25][302/782] Loss_D: 0.6061 Loss_G: 2.7468
[1/25][303/782] Loss_D: 0.7201 Loss_G: 4.0592
[1/25][304/782] Loss_D: 0.7995 Los

[1/25][639/782] Loss_D: 0.6467 Loss_G: 3.3993
[1/25][640/782] Loss_D: 0.6575 Loss_G: 3.6362
[1/25][641/782] Loss_D: 0.4457 Loss_G: 2.7632
[1/25][642/782] Loss_D: 0.5607 Loss_G: 4.3638
[1/25][643/782] Loss_D: 0.4617 Loss_G: 3.0932
[1/25][644/782] Loss_D: 0.6682 Loss_G: 3.0727
[1/25][645/782] Loss_D: 0.4196 Loss_G: 4.6865
[1/25][646/782] Loss_D: 0.2696 Loss_G: 4.2707
[1/25][647/782] Loss_D: 0.3837 Loss_G: 2.9859
[1/25][648/782] Loss_D: 0.4321 Loss_G: 3.4645
[1/25][649/782] Loss_D: 0.5878 Loss_G: 4.4175
[1/25][650/782] Loss_D: 0.6381 Loss_G: 2.7334
[1/25][651/782] Loss_D: 0.4169 Loss_G: 4.3188
[1/25][652/782] Loss_D: 0.5162 Loss_G: 3.2326
[1/25][653/782] Loss_D: 0.6209 Loss_G: 2.7485
[1/25][654/782] Loss_D: 0.5828 Loss_G: 3.7251
[1/25][655/782] Loss_D: 0.4040 Loss_G: 2.8912
[1/25][656/782] Loss_D: 0.3092 Loss_G: 3.4397
[1/25][657/782] Loss_D: 0.3819 Loss_G: 3.7981
[1/25][658/782] Loss_D: 0.6385 Loss_G: 3.7845
[1/25][659/782] Loss_D: 0.4750 Loss_G: 2.5655
[1/25][660/782] Loss_D: 0.6254 Los

...

[22/25][758/782] Loss_D: 0.8350 Loss_G: 3.1044
[22/25][759/782] Loss_D: 0.2230 Loss_G: 2.0852
[22/25][760/782] Loss_D: 0.3814 Loss_G: 3.3332
[22/25][761/782] Loss_D: 0.3403 Loss_G: 3.4184
[22/25][762/782] Loss_D: 0.1987 Loss_G: 3.4308
[22/25][763/782] Loss_D: 0.5527 Loss_G: 1.3123
[22/25][764/782] Loss_D: 0.6504 Loss_G: 4.8514
[22/25][765/782] Loss_D: 0.3243 Loss_G: 3.3071
[22/25][766/782] Loss_D: 0.1667 Loss_G: 3.2681
[22/25][767/782] Loss_D: 0.1556 Loss_G: 3.7677
[22/25][768/782] Loss_D: 0.2267 Loss_G: 2.8046
[22/25][769/782] Loss_D: 0.2916 Loss_G: 5.1657
[22/25][770/782] Loss_D: 0.4537 Loss_G: 2.0090
[22/25][771/782] Loss_D: 0.4612 Loss_G: 5.3975
[22/25][772/782] Loss_D: 0.2146 Loss_G: 4.9003
[22/25][773/782] Loss_D: 0.1362 Loss_G: 3.6384
[22/25][774/782] Loss_D: 0.1346 Loss_G: 3.7759
[22/25][775/782] Loss_D: 0.0953 Loss_G: 4.5962
[22/25][776/782] Loss_D: 0.0879 Loss_G: 4.5386
[22/25][777/782] Loss_D: 0.0441 Loss_G: 4.5607
[22/25][778/782] Loss_D: 0.0583 Loss_G: 4.0590
[22/25][779/7

[23/25][326/782] Loss_D: 0.1798 Loss_G: 3.5342
[23/25][327/782] Loss_D: 0.3988 Loss_G: 5.2288
[23/25][328/782] Loss_D: 1.0470 Loss_G: 0.5462
[23/25][329/782] Loss_D: 1.2444 Loss_G: 6.4582
[23/25][330/782] Loss_D: 1.3204 Loss_G: 1.9226
[23/25][331/782] Loss_D: 0.3561 Loss_G: 2.0828
[23/25][332/782] Loss_D: 0.4487 Loss_G: 4.5835
[23/25][333/782] Loss_D: 0.6143 Loss_G: 1.6722
[23/25][334/782] Loss_D: 0.4015 Loss_G: 3.3136
[23/25][335/782] Loss_D: 0.2727 Loss_G: 4.0585
[23/25][336/782] Loss_D: 0.3397 Loss_G: 2.9234
[23/25][337/782] Loss_D: 0.2399 Loss_G: 3.8560
[23/25][338/782] Loss_D: 0.2608 Loss_G: 3.8419
[23/25][339/782] Loss_D: 0.0837 Loss_G: 3.9343
[23/25][340/782] Loss_D: 0.0677 Loss_G: 4.1496
[23/25][341/782] Loss_D: 0.0613 Loss_G: 4.4468
[23/25][342/782] Loss_D: 0.2536 Loss_G: 5.8453
[23/25][343/782] Loss_D: 0.6220 Loss_G: 3.7769
[23/25][344/782] Loss_D: 0.3737 Loss_G: 4.9904
[23/25][345/782] Loss_D: 0.2552 Loss_G: 3.1284
[23/25][346/782] Loss_D: 0.0777 Loss_G: 3.7870
[23/25][347/7

[23/25][674/782] Loss_D: 0.0745 Loss_G: 4.8943
[23/25][675/782] Loss_D: 0.0276 Loss_G: 5.4217
[23/25][676/782] Loss_D: 0.0303 Loss_G: 5.4002
[23/25][677/782] Loss_D: 0.0559 Loss_G: 5.2448
[23/25][678/782] Loss_D: 0.0412 Loss_G: 4.2109
[23/25][679/782] Loss_D: 0.0145 Loss_G: 5.5247
[23/25][680/782] Loss_D: 0.1431 Loss_G: 6.7135
[23/25][681/782] Loss_D: 0.1139 Loss_G: 7.1261
[23/25][682/782] Loss_D: 0.0721 Loss_G: 4.1739
[23/25][683/782] Loss_D: 0.0273 Loss_G: 4.2021
[23/25][684/782] Loss_D: 0.0296 Loss_G: 5.1048
[23/25][685/782] Loss_D: 0.0753 Loss_G: 6.8710
[23/25][686/782] Loss_D: 0.0250 Loss_G: 6.5898
[23/25][687/782] Loss_D: 0.0055 Loss_G: 7.3098
[23/25][688/782] Loss_D: 0.0303 Loss_G: 6.2057
[23/25][689/782] Loss_D: 0.0099 Loss_G: 5.6799
[23/25][690/782] Loss_D: 0.0073 Loss_G: 6.0569
[23/25][691/782] Loss_D: 0.0198 Loss_G: 5.3417
[23/25][692/782] Loss_D: 0.0255 Loss_G: 5.1958
[23/25][693/782] Loss_D: 0.0104 Loss_G: 5.9021
[23/25][694/782] Loss_D: 0.0258 Loss_G: 5.3326
[23/25][695/7

[24/25][242/782] Loss_D: 0.3268 Loss_G: 3.4705
[24/25][243/782] Loss_D: 0.1488 Loss_G: 4.4651
[24/25][244/782] Loss_D: 0.2290 Loss_G: 3.1085
[24/25][245/782] Loss_D: 0.2601 Loss_G: 5.0315
[24/25][246/782] Loss_D: 0.0934 Loss_G: 4.8842
[24/25][247/782] Loss_D: 0.1270 Loss_G: 3.4803
[24/25][248/782] Loss_D: 0.0893 Loss_G: 3.6772
[24/25][249/782] Loss_D: 0.0910 Loss_G: 4.6844
[24/25][250/782] Loss_D: 0.0447 Loss_G: 4.9387
[24/25][251/782] Loss_D: 0.1163 Loss_G: 3.1694
[24/25][252/782] Loss_D: 0.0831 Loss_G: 4.1881
[24/25][253/782] Loss_D: 0.0849 Loss_G: 4.8892
[24/25][254/782] Loss_D: 0.0726 Loss_G: 4.8363
[24/25][255/782] Loss_D: 0.0651 Loss_G: 4.4631
[24/25][256/782] Loss_D: 0.0298 Loss_G: 4.8618
[24/25][257/782] Loss_D: 0.0994 Loss_G: 3.5072
[24/25][258/782] Loss_D: 0.0965 Loss_G: 4.8362
[24/25][259/782] Loss_D: 0.0435 Loss_G: 5.5446
[24/25][260/782] Loss_D: 0.0809 Loss_G: 4.6624
[24/25][261/782] Loss_D: 0.0574 Loss_G: 4.6180
[24/25][262/782] Loss_D: 0.0205 Loss_G: 4.9655
[24/25][263/7

[24/25][590/782] Loss_D: 0.0372 Loss_G: 4.7783
[24/25][591/782] Loss_D: 0.0344 Loss_G: 5.0561
[24/25][592/782] Loss_D: 0.0273 Loss_G: 5.5639
[24/25][593/782] Loss_D: 0.0301 Loss_G: 5.1879
[24/25][594/782] Loss_D: 0.0165 Loss_G: 6.1914
[24/25][595/782] Loss_D: 0.0202 Loss_G: 5.2190
[24/25][596/782] Loss_D: 0.0342 Loss_G: 5.3134
[24/25][597/782] Loss_D: 0.0200 Loss_G: 5.0227
[24/25][598/782] Loss_D: 0.0179 Loss_G: 5.3069
[24/25][599/782] Loss_D: 0.0372 Loss_G: 5.2339
[24/25][600/782] Loss_D: 0.0246 Loss_G: 5.4733
[24/25][601/782] Loss_D: 0.0297 Loss_G: 5.3599
[24/25][602/782] Loss_D: 0.0309 Loss_G: 6.1392
[24/25][603/782] Loss_D: 0.0112 Loss_G: 5.8588
[24/25][604/782] Loss_D: 0.1160 Loss_G: 7.0124
[24/25][605/782] Loss_D: 0.2996 Loss_G: 7.6424
[24/25][606/782] Loss_D: 4.1113 Loss_G: 0.0055
[24/25][607/782] Loss_D: 6.1876 Loss_G: 8.2767
[24/25][608/782] Loss_D: 5.6580 Loss_G: 1.0198
[24/25][609/782] Loss_D: 1.1965 Loss_G: 1.7405
[24/25][610/782] Loss_D: 1.0486 Loss_G: 3.1201
[24/25][611/7